<a href="https://colab.research.google.com/github/1moi6/pyfuzzy-toolbox/blob/main/notebooks_colab/Aula_3/02_wang_mendel_reta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📏 Wang-Mendel: Exemplo Simples com Reta

**Aula 3 - Minicurso de Sistemas de Inferência Fuzzy**

---

## 🎯 Por que começar com uma reta?

Antes de aproximar funções complexas, vamos ver como Wang-Mendel funciona com o **caso mais simples**: uma função linear!

### Função: f(x) = -2x + 5
- **Domínio**: x ∈ [-1, 1]
- **Inclinação**: -2 (decrescente)
- **Intercepto**: 5
- **Imagem**: y ∈ [3, 7]

### 💡 O que vamos aprender?

1. Como Wang-Mendel **particiona** o espaço
2. Como as **regras fuzzy** são geradas automaticamente
3. Como o sistema faz **aproximação** (não interpolação!)
4. Efeito do **número de partições** na qualidade

---

## 📖 Referência

**Wang, L. X., & Mendel, J. M. (1992)**. "Generating fuzzy rules by learning from examples."  
*IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

---

## 🔧 Instalação

In [ ]:
!pip install pyfuzzy-toolbox[ml] -q

print('✅ pyfuzzy-toolbox instalado com sucesso!')

## 📦 Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fuzzy_systems as fs
from fuzzy_systems.learning import WangMendel

%matplotlib inline

# Configuração
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print('✅ Bibliotecas importadas!')
print(f'   pyfuzzy-toolbox: {fs.__version__}')

## 🔍 Passo 1: Definir o Problema

Função linear: **f(x) = -2x + 5**

In [ ]:
# Função verdadeira
def funcao_real(x):
    """f(x) = -2x + 5"""
    return -2 * x + 5

# Gerar dados de treinamento (com pequeno ruído)
n_samples = 20
noise_level = 0.1

X_train = np.linspace(-1, 1, n_samples)
y_train = funcao_real(X_train) + np.random.normal(0, noise_level, n_samples)

# Dados de teste (grid fino)
X_test = np.linspace(-1, 1, 100)
y_true = funcao_real(X_test)

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='f(x) = -2x + 5 (Real)', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=150, zorder=5, 
           edgecolors='black', linewidths=2, label=f'Dados Observados (n={n_samples})')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Problema: Aproximar reta a partir de dados', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

print(f'📊 Configuração:')
print(f'   • Função: f(x) = -2x + 5')
print(f'   • Domínio: x ∈ [-1, 1]')
print(f'   • Imagem: y ∈ [{funcao_real(1):.1f}, {funcao_real(-1):.1f}]')
print(f'   • Amostras: {n_samples}')
print(f'   • Ruído (σ): {noise_level}')

## 🧠 Passo 2: Criar e Treinar Modelo Wang-Mendel com 5 Partições

In [ ]:
# Criar modelo Wang-Mendel
wm = WangMendel(
    n_partitions=5,
    margin=0.10,
    input_names=['x'],
    output_name='y'
)

print('✅ Modelo Wang-Mendel criado!')
print('   • 5 partições')
print('   • Margem: 10%\n')

# Treinar
wm.fit(X_train.reshape(-1, 1), y_train)

print('\n✅ Treinamento completo!')
print(f'   • Regras geradas: {wm.n_rules}')

## 📊 Passo 3: Visualizar Partições Fuzzy

In [ ]:
# Visualizar partições
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Partições de x
x_universe = np.linspace(wm.fis.inputs['x'].universe[0], wm.fis.inputs['x'].universe[1], 300)
for term_name, term in wm.fis.inputs['x'].terms.items():
    mu = term.membership(x_universe)
    axes[0].plot(x_universe, mu, linewidth=2, label=term_name, alpha=0.7)
    axes[0].fill_between(x_universe, 0, mu, alpha=0.2)

axes[0].set_title('Partições da Entrada x (5 termos)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('Grau de Pertinência', fontsize=12)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([-0.05, 1.05])

# Partições de y
y_universe = np.linspace(wm.fis.outputs['y'].universe[0], wm.fis.outputs['y'].universe[1], 300)
for term_name, term in wm.fis.outputs['y'].terms.items():
    mu = term.membership(y_universe)
    axes[1].plot(y_universe, mu, linewidth=2, label=term_name, alpha=0.7)
    axes[1].fill_between(y_universe, 0, mu, alpha=0.2)

axes[1].set_title('Partições da Saída y (5 termos)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('y', fontsize=12)
axes[1].set_ylabel('Grau de Pertinência', fontsize=12)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([-0.05, 1.05])

plt.tight_layout()
plt.show()

print('✅ Partições visualizadas!')

## 📜 Passo 4: Inspecionar Regras Geradas

In [ ]:
print('='*70)
print('📜 REGRAS GERADAS AUTOMATICAMENTE')
print('='*70)
print(f'\nTotal de regras: {len(wm.fis.rules)}\n')

for i, rule in enumerate(wm.fis.rules, 1):
    # Extrair antecedentes e consequentes
    ant_str = ', '.join([f'{var} é {term}' for var, term in rule.antecedents.items()])
    cons_str = ', '.join([f'{var} é {term}' for var, term in rule.consequent.items()])
    print(f'   {i}. SE {ant_str} ENTÃO {cons_str}')

print('\n' + '='*70)
print('💡 Regras capturam a relação linear entre x e y!')
print('='*70)

## 🧪 Passo 5: Testar o Sistema

In [ ]:
# Fazer predições
y_pred = wm.predict(X_test.reshape(-1, 1))

# Calcular métricas
mse = np.mean((y_pred - y_true) ** 2)
mae = np.mean(np.abs(y_pred - y_true))
erro_max = np.max(np.abs(y_pred - y_true))
r2 = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

print('='*70)
print('📊 MÉTRICAS DE DESEMPENHO')
print('='*70)
print(f'   • MSE (Erro Médio Quadrático):      {mse:.6f}')
print(f'   • MAE (Erro Absoluto Médio):        {mae:.6f}')
print(f'   • Erro Máximo:                      {erro_max:.6f}')
print(f'   • R² (Coeficiente de Determinação): {r2:.6f}')
print('='*70)

# Visualizar aproximação
plt.figure(figsize=(12, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='Função Real (f(x) = -2x + 5)', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=150, zorder=5, 
           edgecolors='black', linewidths=2, label='Dados de Treinamento', alpha=0.9)
plt.plot(X_test, y_pred, 'b-', linewidth=3, 
         label=f'Wang-Mendel (5 partições, {wm.n_rules} regras)', alpha=0.8)
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title(f'Aproximação com Wang-Mendel | MSE = {mse:.6f} | R² = {r2:.4f}', 
         fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Análise do erro
plt.figure(figsize=(12, 5))
erro_abs = np.abs(y_pred - y_true)
plt.plot(X_test, erro_abs, 'r-', linewidth=2.5)
plt.fill_between(X_test, 0, erro_abs, alpha=0.3, color='red')
plt.xlabel('x', fontsize=13)
plt.ylabel('Erro Absoluto |y_real - y_predito|', fontsize=13)
plt.title(f'Análise do Erro de Aproximação | MAE = {mae:.6f}', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 🔍 Passo 6: Exemplo de Inferência Detalhada

Vamos ver **passo a passo** como o sistema faz uma predição!

In [ ]:
# Escolher um ponto para análise
x_teste = 0.5
y_real = funcao_real(x_teste)

print('='*70)
print(f'🔍 ANÁLISE DETALHADA PARA x = {x_teste}')
print('='*70)
print(f'📌 Valor real: y = f({x_teste}) = -2({x_teste}) + 5 = {y_real:.3f}')
print('='*70)
print()

# Fuzzificação
graus_x = wm.fis.inputs['x'].fuzzify(x_teste)
print('[ETAPA 1] FUZZIFICAÇÃO')
print('-'*70)
print(f'x = {x_teste}')
for termo, mu in graus_x.items():
    if mu > 0:
        print(f'  μ({termo}) = {mu:.3f}')

# Fazer predição
y_predito = wm.predict([[x_teste]])[0]

print()
print('='*70)
print(f'✅ Resultado: y_predito = {y_predito:.4f}')
print(f'✅ Valor real: y_real = {y_real:.4f}')
print(f'✅ Erro: |{y_predito:.4f} - {y_real:.4f}| = {abs(y_predito - y_real):.4f}')
print('='*70)

## 🔬 Passo 7: Experimento - Efeito do Número de Partições

Vamos testar com **3, 5, 7 e 11 partições** para ver o efeito!

In [ ]:
print('🔬 Testando diferentes números de partições...\n')

particoes_teste = [3, 5, 7, 11]
resultados = {}

for n_part in particoes_teste:
    print(f'▶️  Testando com {n_part} partições...')
    
    # Criar modelo
    wm_exp = WangMendel(
        n_partitions=n_part,
        margin=0.10,
        input_names=['x'],
        output_name='y'
    )
    
    # Treinar
    wm_exp.fit(X_train.reshape(-1, 1), y_train)
    
    # Predições
    y_pred_exp = wm_exp.predict(X_test.reshape(-1, 1))
    mse_exp = np.mean((y_pred_exp - y_true) ** 2)
    r2_exp = 1 - (np.sum((y_true - y_pred_exp)**2) / np.sum((y_true - np.mean(y_true))**2))
    
    resultados[n_part] = {
        'y_pred': y_pred_exp,
        'mse': mse_exp,
        'r2': r2_exp,
        'n_regras': wm_exp.n_rules
    }
    
    print(f'   ✅ MSE = {mse_exp:.6f} | R² = {r2_exp:.4f} | {resultados[n_part]["n_regras"]} regras\n')

print('='*70)
print('📊 Resumo:')
for n_part in particoes_teste:
    print(f'   {n_part:2d} partições: MSE = {resultados[n_part]["mse"]:.6f} | R² = {resultados[n_part]["r2"]:.4f} | {resultados[n_part]["n_regras"]:2d} regras')
print('='*70)

## 📈 Visualizar Comparação

In [ ]:
# Gráfico comparativo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Aproximações
ax1.plot(X_test, y_true, 'k-', linewidth=3, label='Real', alpha=0.7)
ax1.scatter(X_train, y_train, color='red', s=100, zorder=5, 
           edgecolors='black', linewidths=2, label='Dados', alpha=0.8)

cores = ['blue', 'green', 'purple', 'orange']
for (n_part, cor) in zip(particoes_teste, cores):
    ax1.plot(X_test, resultados[n_part]['y_pred'], 
            linestyle='--', linewidth=2, color=cor,
            label=f'{n_part} partições (MSE={resultados[n_part]["mse"]:.4f})', 
            alpha=0.8)

ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel('y', fontsize=12)
ax1.set_title('Comparação: Efeito do Número de Partições', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Gráfico 2: MSE vs Partições
mses = [resultados[n]['mse'] for n in particoes_teste]
bars = ax2.bar(particoes_teste, mses, color=cores, alpha=0.7, edgecolor='black', linewidth=2)

for bar, mse, n_part in zip(bars, mses, particoes_teste):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + height*0.05,
            f'{mse:.5f}\n{resultados[n_part]["n_regras"]} regras',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

ax2.set_xlabel('Número de Partições', fontsize=12)
ax2.set_ylabel('MSE', fontsize=12)
ax2.set_title('MSE vs Número de Partições', fontsize=13, fontweight='bold')
ax2.set_xticks(particoes_teste)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('\n💡 Observação:')
print('   Mais partições geralmente = menor erro, mas mais regras!')
print('   Trade-off entre precisão e complexidade.')

---

## 🎓 Conclusões

### ✅ O que aprendemos?

1. **Particionamento Fuzzy**
   - Divide o domínio em regiões linguísticas (Baixo, Médio, Alto)
   - Cada ponto pode pertencer a múltiplas regiões (sobreposição)

2. **Geração Automática de Regras**
   - Regras são extraídas diretamente dos dados
   - Cada dado gera uma regra candidata
   - Conflitos são resolvidos mantendo regra com maior grau

3. **Aproximação vs Interpolação**
   - Wang-Mendel **aproxima** (não interpola)
   - Não passa exatamente pelos pontos de dados
   - Robusto a ruído!

4. **Trade-off: Partições vs Complexidade**
   - Mais partições → menor erro
   - Mais partições → mais regras → mais complexo
   - Para n entradas e p partições: até p^n regras!

### 🎯 Wang-Mendel para uma Reta:

- ✅ Funciona bem mesmo para funções simples
- ✅ Gera regras interpretáveis
- ✅ Número de partições afeta precisão
- ✅ Margem no universo evita problemas com dados de teste

---

### 🚀 Próximos Passos:

Agora que entendemos o básico com uma reta, vamos para:
- **Funções não-lineares**: `01_wang_mendel_didatico.ipynb` (sin(x) + 0.1x)
- **Classificação**: `03_wang_mendel_iris.ipynb` (Iris dataset)

---

### 📚 Recursos

- **PyPI**: https://pypi.org/project/pyfuzzy-toolbox/
- **GitHub**: https://github.com/1moi6/pyfuzzy-toolbox

### 📖 Referência:

Wang, L. X., & Mendel, J. M. (1992). "Generating fuzzy rules by learning from examples."  
*IEEE Transactions on Systems, Man, and Cybernetics*, 22(6), 1414-1427.

---

*Notebook desenvolvido para o Minicurso de Sistemas de Inferência Fuzzy - Aula 3 - 2025*  
*Usando pyfuzzy-toolbox v1.0.0*